In [1]:
# hyper parameter tuning
# using Unet, loss = mse

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, glob
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # GPU
import tensorflow.python as tf

import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from scipy import signal, ndimage
from scipy.io import loadmat, savemat
from skimage import data, img_as_float, exposure
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

In [3]:
def uniform_circle(x0, y0, sigma, width, height):
    x = np.arange(0, width, 1, float)
    y = np.arange(0, height, 1, float)[:, np.newaxis]
    tmp = (x - x0)**2 + (y - y0)**2 < sigma ** 2
    return tmp.astype(int)

In [4]:
def get_y_as_heatmap_u(y, width, height, sigma=3):
    ymap = []
    for i in range(y.shape[0]):
        msk_array = np.zeros((width, height, 15), dtype=np.float32)
        for j in range(15):
            msk = uniform_circle(y[i, 2*j]*width, y[i, 2*j+1]*height, sigma, width, height)
            msk_array[:,:,j] = msk
        ymap.append(msk_array)
    ymap = np.array(ymap)
    return ymap

In [ ]:
def gaussian_k(x0, y0, sigma, width, height):
    x = np.arange(0, width, 1, float)
    y = np.arange(0, height, 1, float)[:, np.newaxis]
    return np.exp(-((x-x0)**2 + (y-y0)**2) / (2*sigma**2))

def get_y_as_heatmap_k(y, width, height, sigma=3):
    ymap = []
    for i in range(y.shape[0]):
        msk_array = np.zeros((width, height, 15), dtype=np.float32)
        for j in range(15):
            msk = gaussian_k(y[i, 2*j]*width, y[i, 2*j+1]*height, sigma, width, height)
            msk_array[:,:,j] = msk
        ymap.append(msk_array)
    ymap = np.array(ymap)
    return ymap

In [5]:
from func_processing import img_affine, img_hflip, load, get_avg_xy
from sklearn.model_selection import train_test_split

FTRAIN = './data/facial-keypoints-detection/training/training.csv'

In [6]:
# load data
X, Y = load(FTRAIN)
X = X.reshape(-1,96,96)
# split into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [7]:
# training data augmentation
np.random.seed(0)
dz = np.random.randint(-5,5,(X_train.shape[0],2))
X_train0 = []
Y_train0 = []
for i, x in enumerate(X_train):
    xt, yt = img_affine(np.squeeze(x), Y_train[i], dz[i,0], dz[i,1], 96, 96)
    X_train0.append(xt)
    Y_train0.append(yt)
    xf, yf = img_hflip(np.squeeze(x), Y_train[i])
    X_train0.append(xf)
    Y_train0.append(yf)
X_train0 = np.array(X_train0)
Y_train0 = np.array(Y_train0)

In [8]:
X_train0 = np.expand_dims(X_train0, axis=3)

In [9]:
from func_UNET import unet
SIGMA = [5, 7, 9, 11]
DROPOUT = [0.1, 0.5]
NFILTER = [8, 16, 32]
ERR = np.zeros((4, 2, 3))
MIN_ERR = np.inf
for p,sigma in enumerate(SIGMA):
    for q,dropout in enumerate(DROPOUT):
        for s,n in enumerate(NFILTER):
            # y_train0 = get_y_as_heatmap_u(Y_train0, 96, 96, sigma)
            y_train0 = get_y_as_heatmap_k(Y_train0, 96, 96, sigma)
            X_train, X_valid, y_train, y_valid = train_test_split(X_train0, y_train0, test_size=0.2, random_state=43)

            model = unet(96, 96, 1, DROPOUT=dropout, NFILTER=n)
            model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

            from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
            callbacks = [
            EarlyStopping(patience=10, verbose=1),
            ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
            ModelCheckpoint('model-tgs-seg2.h5', verbose=1, save_best_only=True, save_weights_only=True)
            ]

            history = model.fit(X_train, y_train, batch_size=32, epochs=100, callbacks=callbacks, validation_data=(X_valid, y_valid))

            plt.plot(history.history['loss'], label='traning loss')
            plt.plot(history.history['val_loss'], label = 'validation loss')
            plt.yscale('log')
            plt.legend()
            plt.xlabel('Epoch')
            plt.ylabel('loss')
            plt.savefig('./model/hp2/hist_sigma%d_dropout%.1f_n%d.png'%(sigma, dropout, n))
            plt.close()

            model_predict = model.predict(X_test)
            y_pred = np.zeros((model_predict.shape[0], 30))
            for i in range(model_predict.shape[0]):
                for j in range(15):
                    msk = np.squeeze(model_predict[i,:,:,j])
                    yx, prob = get_avg_xy(msk, 25)
                    y_pred[i,2*j] = yx[1] / 96
                    y_pred[i,2*j+1] = yx[0] / 96
            error = np.linalg.norm(Y_test - y_pred)
            ERR[p,q,s] = error
            np.savetxt('unet_hp.txt',(sigma,dropout,n,error),fmt='.1f',delimiter=',')
            if error < MIN_ERR:
                MIN_ERR = error
                MIN_SIGMA = sigma
                MIN_DROPOUT = dropout
                MIN_N = n               

- acc: 0.1254 - val_loss: 0.0037 - val_acc: 0.1123
Epoch 77/100
3040/3081 [============================>.] - ETA: 0s - loss: 0.0042 - acc: 0.1249
Epoch 00077: val_loss improved from 0.00367 to 0.00366, saving model to model-tgs-seg2.h5
3081/3081 [==============================] - 4s 1ms/sample - loss: 0.0042 - acc: 0.1249 - val_loss: 0.0037 - val_acc: 0.1117
Epoch 78/100
3040/3081 [============================>.] - ETA: 0s - loss: 0.0042 - acc: 0.1245
Epoch 00078: val_loss did not improve from 0.00366
3081/3081 [==============================] - 4s 1ms/sample - loss: 0.0042 - acc: 0.1244 - val_loss: 0.0037 - val_acc: 0.1119
Epoch 79/100
3040/3081 [============================>.] - ETA: 0s - loss: 0.0043 - acc: 0.1245
Epoch 00079: val_loss did not improve from 0.00366
3081/3081 [==============================] - 4s 1ms/sample - loss: 0.0043 - acc: 0.1244 - val_loss: 0.0037 - val_acc: 0.1124
Epoch 80/100
3040/3081 [============================>.] - ETA: 0s - loss: 0.0042 - acc: 0.1245
Ep

KeyboardInterrupt: 

In [ ]:
# sigma = 5, N = 32 * 2